### Quick Start

#### Preprocessing

In [1]:
import ray

In [2]:
# Load data
dataset = ray.data.read_csv("s3://anonymous@air-example-data/breast_cancer.csv")

# Split data into train and validation
train_dataset, valid_dataset = dataset.train_test_split(test_size=0.3)

# Create a test dataset by dropping the target column
test_dataset = valid_dataset.drop_columns(cols=['target'])

2022-10-16 22:31:19,002	INFO worker.py:1518 -- Started a local Ray instance.
2022-10-16 22:31:36,926	WARNING read_api.py:291 -- ⚠️  The number of blocks in this dataset (1) limits its parallelism to 1 concurrent tasks. This is much less than the number of available CPU slots in the cluster. Use `.repartition(n)` to increase the number of dataset blocks.
Map_Batches: 100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  5.22it/s]


#### XGBoost

In [11]:
from ray.data.preprocessors import StandardScaler
from ray.air.config import ScalingConfig
from ray.train.xgboost import XGBoostTrainer

# Preprocessing
preprocessor = StandardScaler(columns=["mean radius", "mean texture"])

# Training
trainer = XGBoostTrainer(
    scaling_config=ScalingConfig(
        num_workers=4,    # number of workers to use for data parallelism
        use_gpu=False,    # Whether to use GPU accleration
    ),
    label_column='target',
    num_boost_round=20,
    params={
        'objective': 'binary:logistic',    # XGBoost specific params
        'eval_metric': ['logloss', 'error']    # "tree_method": "gpu_hist"
    },
    datasets={'train': train_dataset, 'valid': valid_dataset},
    preprocessor=preprocessor,
)

result = trainer.fit()
print(result.metrics)

2022-10-16 22:43:34,371	INFO tensorboardx.py:170 -- pip install "ray[tune]" to see TensorBoard files.
2022-10-16 22:43:34,373	WARNING callback.py:108 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


Trial name,status,loc,iter,total time (s),train-logloss,train-error,valid-logloss
XGBoostTrainer_87799_00000,TERMINATED,127.0.0.1:42146,21,15.3273,0.0179314,0,0.0879959


(_RemoteRayXGBoostActor pid=42173) [22:43:50] task [xgboost.ray]:140413578906544 got new rank 0
(_RemoteRayXGBoostActor pid=42175) [22:43:50] task [xgboost.ray]:140233097997328 got new rank 1
(_RemoteRayXGBoostActor pid=42191) [22:43:50] task [xgboost.ray]:140477513718752 got new rank 2
(_RemoteRayXGBoostActor pid=42192) [22:43:50] task [xgboost.ray]:140655252605824 got new rank 3


Result for XGBoostTrainer_87799_00000:
  date: 2022-10-16_22-43-53
  done: false
  experiment_id: b3191d18ca96442990797af779089a50
  hostname: YONGJINs-MacBook-Pro.local
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 42146
  time_since_restore: 14.711936950683594
  time_this_iter_s: 14.711936950683594
  time_total_s: 14.711936950683594
  timestamp: 1665927833
  timesteps_since_restore: 0
  train-error: 0.02512562814070352
  train-logloss: 0.46656074401122244
  training_iteration: 1
  trial_id: '87799_00000'
  valid-error: 0.11695906432748537
  valid-logloss: 0.502969495385711
  warmup_time: 0.005882978439331055
  
Result for XGBoostTrainer_87799_00000:
  date: 2022-10-16_22-43-53
  done: true
  experiment_id: b3191d18ca96442990797af779089a50
  experiment_tag: '0'
  hostname: YONGJINs-MacBook-Pro.local
  iterations_since_restore: 21
  node_ip: 127.0.0.1
  pid: 42146
  time_since_restore: 15.327291011810303
  time_this_iter_s: 0.47252488136291504
  time_total_s: 15.32729101181

2022-10-16 22:43:54,021	INFO tune.py:758 -- Total run time: 19.66 seconds (19.52 seconds for the tuning loop).


{'train-logloss': 0.01793136571220417, 'train-error': 0.0, 'valid-logloss': 0.0879958809778225, 'valid-error': 0.04093567251461988, 'time_this_iter_s': 0.47252488136291504, 'done': True, 'timesteps_total': None, 'episodes_total': None, 'training_iteration': 21, 'trial_id': '87799_00000', 'experiment_id': 'b3191d18ca96442990797af779089a50', 'date': '2022-10-16_22-43-53', 'timestamp': 1665927833, 'time_total_s': 15.327291011810303, 'pid': 42146, 'hostname': 'YONGJINs-MacBook-Pro.local', 'node_ip': '127.0.0.1', 'config': {}, 'time_since_restore': 15.327291011810303, 'timesteps_since_restore': 0, 'iterations_since_restore': 21, 'warmup_time': 0.005882978439331055, 'experiment_tag': '0'}


In [12]:
from ray import tune
from ray.tune.tuner import Tuner, TuneConfig
from ray.air.config import RunConfig

In [13]:
# Hyperparmeter Tuning
param_space = {'params': {'max_depth': tune.randint(1, 9)}}
metric = 'train-logloss'

In [14]:
tuner = Tuner(
    trainer,
    param_space=param_space,
    tune_config=TuneConfig(num_samples=5, metric=metric, mode='min')
)

# Excute tuning
result_grid = tuner.fit()

# Fetch the best result
best_result = result_grid.get_best_result()
print(f"Best Result: {best_result}")

2022-10-16 22:48:04,159	WARNING callback.py:108 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


Trial name,status,loc,params/max_depth,iter,total time (s),train-logloss,train-error,valid-logloss
XGBoostTrainer_28481_00000,TERMINATED,127.0.0.1:43130,2,21,12.3457,0.0410722,0.00502513,0.1029
XGBoostTrainer_28481_00001,TERMINATED,127.0.0.1:43203,5,21,13.6486,0.0185925,0,0.0818791
XGBoostTrainer_28481_00002,TERMINATED,127.0.0.1:43261,1,21,13.8138,0.0954546,0.0201005,0.11149
XGBoostTrainer_28481_00003,TERMINATED,127.0.0.1:43319,5,21,13.8127,0.0185925,0,0.0818791
XGBoostTrainer_28481_00004,TERMINATED,127.0.0.1:43385,2,21,13.7265,0.0410722,0.00502513,0.1029


(_RemoteRayXGBoostActor pid=43161) [22:48:18] task [xgboost.ray]:140566358526800 got new rank 1
(_RemoteRayXGBoostActor pid=43158) [22:48:18] task [xgboost.ray]:140395063147392 got new rank 0
(_RemoteRayXGBoostActor pid=43175) [22:48:18] task [xgboost.ray]:140267852003840 got new rank 2
(_RemoteRayXGBoostActor pid=43176) [22:48:18] task [xgboost.ray]:140621523053296 got new rank 3


Result for XGBoostTrainer_28481_00000:
  date: 2022-10-16_22-48-19
  done: false
  experiment_id: e6f83f23135245dabe3aec23525d5504
  hostname: YONGJINs-MacBook-Pro.local
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 43130
  time_since_restore: 11.232172966003418
  time_this_iter_s: 11.232172966003418
  time_total_s: 11.232172966003418
  timestamp: 1665928099
  timesteps_since_restore: 0
  train-error: 0.04522613065326633
  train-logloss: 0.48458515279856157
  training_iteration: 1
  trial_id: '28481_00000'
  valid-error: 0.1111111111111111
  valid-logloss: 0.5170963152110228
  warmup_time: 0.005660057067871094
  
Result for XGBoostTrainer_28481_00000:
  date: 2022-10-16_22-48-20
  done: true
  experiment_id: e6f83f23135245dabe3aec23525d5504
  experiment_tag: 0_max_depth=2
  hostname: YONGJINs-MacBook-Pro.local
  iterations_since_restore: 21
  node_ip: 127.0.0.1
  pid: 43130
  time_since_restore: 12.345662832260132
  time_this_iter_s: 0.6764390468597412
  time_total_s: 12.34

(_RemoteRayXGBoostActor pid=43220) [22:48:35] task [xgboost.ray]:140322445052944 got new rank 0
(_RemoteRayXGBoostActor pid=43221) [22:48:35] task [xgboost.ray]:140210994023152 got new rank 1
(_RemoteRayXGBoostActor pid=43243) [22:48:35] task [xgboost.ray]:140330614512752 got new rank 3
(_RemoteRayXGBoostActor pid=43242) [22:48:35] task [xgboost.ray]:140679429614608 got new rank 2


...

Result for XGBoostTrainer_28481_00003:
  date: 2022-10-16_22-49-14
  done: false
  experiment_id: bef307c5a147406f81a2e9f0fc2dee87
  hostname: YONGJINs-MacBook-Pro.local
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 43319
  time_since_restore: 13.43497610092163
  time_this_iter_s: 13.43497610092163
  time_total_s: 13.43497610092163
  timestamp: 1665928154
  timesteps_since_restore: 0
  train-error: 0.02512562814070352
  train-logloss: 0.4670509481849383
  training_iteration: 1
  trial_id: '28481_00003'
  valid-error: 0.09941520467836257
  valid-logloss: 0.501814771813956
  warmup_time: 0.005287885665893555
  
Result for XGBoostTrainer_28481_00003:
  date: 2022-10-16_22-49-15
  done: true
  experiment_id: bef307c5a147406f81a2e9f0fc2dee87
  experiment_tag: 3_max_depth=5
  hostname: YONGJINs-MacBook-Pro.local
  iterations_since_restore: 21
  node_ip: 127.0.0.1
  pid: 43319
  time_since_restore: 13.81273603439331
  time_this_iter_s: 0.2525599002838135
  time_total_s: 13.8127360

(_RemoteRayXGBoostActor pid=43402) [22:49:29] task [xgboost.ray]:140162951408560 got new rank 1
(_RemoteRayXGBoostActor pid=43401) [22:49:29] task [xgboost.ray]:140440858651568 got new rank 0
(_RemoteRayXGBoostActor pid=43416) [22:49:29] task [xgboost.ray]:140216202292240 got new rank 2
(_RemoteRayXGBoostActor pid=43417) [22:49:29] task [xgboost.ray]:140379561003712 got new rank 3


Result for XGBoostTrainer_28481_00004:
  date: 2022-10-16_22-49-32
  done: false
  experiment_id: cc725e3c8ab04d1a9c2cdede03203dbd
  hostname: YONGJINs-MacBook-Pro.local
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 43385
  time_since_restore: 13.363611936569214
  time_this_iter_s: 13.363611936569214
  time_total_s: 13.363611936569214
  timestamp: 1665928172
  timesteps_since_restore: 0
  train-error: 0.04522613065326633
  train-logloss: 0.48458515279856157
  training_iteration: 1
  trial_id: '28481_00004'
  valid-error: 0.1111111111111111
  valid-logloss: 0.5170963152110228
  warmup_time: 0.005262136459350586
  
Result for XGBoostTrainer_28481_00004:
  date: 2022-10-16_22-49-32
  done: true
  experiment_id: cc725e3c8ab04d1a9c2cdede03203dbd
  experiment_tag: 4_max_depth=2
  hostname: YONGJINs-MacBook-Pro.local
  iterations_since_restore: 21
  node_ip: 127.0.0.1
  pid: 43385
  time_since_restore: 13.726532936096191
  time_this_iter_s: 0.23717474937438965
  time_total_s: 13.7

2022-10-16 22:49:33,020	INFO tune.py:758 -- Total run time: 88.87 seconds (88.73 seconds for the tuning loop).


Best Result: Result(metrics={'train-logloss': 0.01859253883974898, 'train-error': 0.0, 'valid-logloss': 0.08187905736866663, 'valid-error': 0.02339181286549707, 'done': True, 'trial_id': '28481_00001', 'experiment_tag': '1_max_depth=5'}, error=None, log_dir=PosixPath('/Users/yjkim/ray_results/XGBoostTrainer_2022-10-16_22-48-04/XGBoostTrainer_28481_00001_1_max_depth=5_2022-10-16_22-48-22'))


In [15]:
from ray.train.batch_predictor import BatchPredictor
from ray.train.xgboost import XGBoostPredictor

In [16]:
# Batch Inference

# also create a checkpoint from a trained model using `XGBoostCheckpoint.from_model`
checkpoint = best_result.checkpoint

batch_predictor = BatchPredictor.from_checkpoint(checkpoint, XGBoostPredictor)

predicted_probabilities = batch_predictor.predict(test_dataset)
predicted_probabilities.show()

Map Progress (1 actors 1 pending): 100%|███████████████████████| 1/1 [00:02<00:00,  2.03s/it]

{'predictions': 0.9967517852783203}
{'predictions': 0.9956080317497253}
{'predictions': 0.0035305859055370092}
{'predictions': 0.9967517852783203}
{'predictions': 0.9968827962875366}
{'predictions': 0.9960047602653503}
{'predictions': 0.9917015433311462}
{'predictions': 0.995203971862793}
{'predictions': 0.27522677183151245}
{'predictions': 0.9821683764457703}
{'predictions': 0.0035305859055370092}
{'predictions': 0.9960752129554749}
{'predictions': 0.9656598567962646}
{'predictions': 0.9889512658119202}
{'predictions': 0.9943472743034363}
{'predictions': 0.26353344321250916}
{'predictions': 0.4275687336921692}
{'predictions': 0.9949140548706055}
{'predictions': 0.9823238849639893}
{'predictions': 0.0035305859055370092}
